In [1]:
# Input parameters
model_patch_size = 8
verbose          = 0
reset_kernel     = False

In [2]:
# Imports
from dvats.all import *
from tsai.data.preparation import SlidingWindow
from fastcore.all import *
import wandb
wandb_api = wandb.Api()
from yaml import load, FullLoader
import dvats.utils as ut
from dvats.imports import beep
import pandas as pd
from copy import deepcopy
import matplotlib.pyplot as plt
import ydata_profiling as ydp
import seaborn as sns

/usr/local/share/miniconda3/envs/env/lib/python3.10/site-packages/aeon/base/__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(
Using the latest cached version of the module from /home/macu/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mse/5303f8acce26cdffcedb23101b42d1660bf34c3d8885adcd0bb262916312a51a (last modified on Mon Jan 27 15:41:53 2025) since it couldn't be found locally at evaluate-metric--mse, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/macu/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mse/5303f8acce26cdffcedb23101b42d1660bf34c3d8885adcd0bb2629


Octave is ready <oct2py.core.Oct2Py object at 0x7f378977d0f0>

















In [24]:
import torch
torch.cuda.set_device(1)

In [4]:
from fastai.losses import MSELossFlat
from dvats.encoder import MAELossFlat, EvalMSE, EvalRMSE, EvalMAE, EvalSMAPE
import dvats.config as cfg_
import gc
import os

In [5]:
# User and project
entity = os.environ.get("WANDB_ENTITY")
project = os.environ.get("WANDB_PROJECT")
folder = entity+'/'+project+'/'

# Dataset
dataset = 'gtrends_kohls'
dataset_version = 'v2'
enc_artifact_dataset = folder + dataset + ':' + dataset_version

# Models
model_family = 'zeroshot-moment'
task = 'embedding'
enc_artifact_small_name = folder + model_family + '-small-' + task + ':v0'
enc_artifact_base_name  =  folder + model_family + '-base-' + task + ':v0'
enc_artifact_large_name = folder + model_family + '-large-' + task + ':v0'

In [6]:
print("Getting dataset artifact: ", enc_artifact_dataset)
df_artifact = wandb_api.artifact(enc_artifact_dataset, type = 'dataset')

Getting dataset artifact:  mi-santamaria/deepvats/gtrends_kohls:v2


In [7]:
print(df_artifact.name)
df = df_artifact.to_df()
display(df.head())
df.shape

gtrends_kohls:v2


wandb:   1 of 1 files downloaded.  


,volume
2004-01-01,0.010417
2004-01-08,0.010417
2004-01-15,0.010417
2004-01-22,0.000000
2004-01-29,0.000000


(440, 1)

In [8]:
enc_input, _ = SlidingWindow(window_len=17, stride=2, get_y=[])(df)
enc_input.shape

(212, 1, 17)

In [11]:
%cd work/nbs_pipeline

/home/macu/work/nbs_pipeline


In [12]:
user, project, version, data, config, job_type = cfg_.get_artifact_config_MVP(False)

In [25]:
common_args = {
    "X": df,
    "stride": 1,
    "batch_size": config['batch_size'],
    "cpu": False,
    "to_numpy": False,
    "time_flag": True,
    "n_windows": None,
    "n_windows_percent": None,
    "shot": True,
    "eval_pre": True,
    "eval_post": True,
    "lr": config['r'], #use enc_run lr,
    "lr_scheduler_flag": False,
    "lr_scheduler_name": "cosine_with_restarts",
    "lr_scheduler_num_warmup_steps": None,
    "window_sizes": [12],
    "full_dataset": True,
    "window_sizes_offset": 0.05,
    "windows_min_distance": 0, #int(np.ceil(1.5*enc_input.shape[0]/100)),
    "print_to_path": True,
    "print_path": "./logs_aux2.txt",
    "print_mode": "w",
    "use_moment_masks": True,
    "mask_stateful": config['mask_stateful'],
    "mask_future": config['mask_future'],
    "mask_sync": config['mask_sync'],
    "analysis_mode": config['analysis_mode'],
    "use_wandb": config['use_wandb'],
    "norm_by_sample": config['norm_by_sample'],
    "norm_use_single_batch": config['norm_use_single_batch'],
    "show_plot": True,
    "metrics": [EvalMSE, EvalRMSE, EvalMAE, EvalSMAPE],
    "metrics_args": [{'squared': False}, {'squared': True}, {}, {}],
    "metrics_names":["mse", "rmse", "mae", "smape"],
    "metrics_dict": None,
    "criterion": torch.nn.MSELoss()
}

In [26]:
def cases_loop(
    model, 
    n_epochs_list, 
    dataset_percents, 
    maskared_percents, 
    n_sizes_list, 
    summarized = True, 
    do_beep = True, 
    verbose = 1,
    save = True,
    file_errors = "",
    file_results = ""
):
    mssg = ut.Mssg(verbose = verbose, level = -1)
    result_columns = [
        'model_size','n_epochs','dataset_percent','maskared_percent','n_windows', 
        'time',
        'first_train_loss','first_mse','first_rmse','first_mae','first_smape', 
        'last_train_loss','last_mse','last_rmse','last_mae','last_smape'
    ]
    result_columns = result_columns if summarized else result_columns + ['losses','eval_results_pre','eval_results_post']
    results = pd.DataFrame(columns = result_columns)
    
    errors = pd.DataFrame(
        columns = [
            'model_size',
            'n_epochs',
            'dataset_percent',
            'maskared_percent',
            'n_windows',
            'windows',
            'error'
        ]
    )
    model_backup = deepcopy(model)
    i = 0
    for n_epochs in n_epochs_list:
        for dataset_percent in dataset_percents:
            print(dataset_percent)
            for maskared_percent in maskared_percents:
                #model.mask_generator = Masking(mask_ratio = maskared_percent)
                for sizes in n_sizes_list:
                    print(f"--> epoch {n_epochs}, dataset_percent {dataset_percent}, mask {maskared_percent}")
                    print(f" sizes {sizes}")
                    print(f"Cuda memmory allocated: {torch.cuda.memory_allocated()}")
                    model_case = deepcopy(model_backup)
                    case = {
                            'model_size': "small",
                            'n_epochs': n_epochs,
                            'dataset_percent': dataset_percent,
                            'maskared_percent': maskared_percent,
                            'n_windows': sizes,
                            'windows': None
                           }
                    result_dict = deepcopy(case)
                    error_dict = deepcopy(case)
                    error = False
                    print(1-dataset_percent)
                    result = fine_tune(
                        enc_learn           = model_case,
                        window_mask_percent = maskared_percent,
                        training_percent    = dataset_percent,
                        validation_percent  = 1-dataset_percent,
                        num_epochs          = n_epochs,
                        n_window_sizes      = sizes,
                        verbose             = verbose,
                        register_errors     = True,
                        save_best_or_last   = True, # only available for moment
                        **common_args    
                    )
                    try:
                        internal_errors = result[10]
                    except:
                        internal_errors = pd.DataFrame(columns=["window", "error"])
                    if len(result[0]) > 0:
                        result_dict.update({
                            'time'             : result[4],
                            'windows'          : result[8].cpu() if isinstance(result[8], torch.Tensor) else result[8],
                            'first_train_loss' : result[0][0][0].cpu().item() if torch.is_tensor(result[0][0][0]) else result[0][0][0],
                            'last_train_loss'  : result[0][-1][-1].cpu().item() if torch.is_tensor(result[0][-1][-1]) else result[0][-1][-1],
                            'first_eval_loss'  : result[1]['loss'].cpu().item() if torch.is_tensor(result[1]['loss']) else result[1]['loss'],
                            'last_eval_loss'   : result[2]['loss'][-1].cpu().item() if torch.is_tensor(result[2]['loss'][-1]) else result[2]['loss'][-1],
                            'first_mse'        : result[1]['mse'].cpu().item() if torch.is_tensor(result[1]['mse']) else result[1]['mse'],
                            'last_mse'         : result[2]['mse'][-1].cpu().item() if torch.is_tensor(result[2]['mse'][-1]) else result[2]['mse'][-1],
                            'first_rmse'       : result[1]['rmse'].cpu().item() if torch.is_tensor(result[1]['rmse']) else result[1]['rmse'],
                            'last_rmse'        : result[2]['rmse'][-1].cpu().item() if torch.is_tensor(result[2]['rmse'][-1]) else result[2]['rmse'][-1],
                            'first_mae'        : result[1]['mae'].cpu().item() if torch.is_tensor(result[1]['mae']) else result[1]['mae'],
                            'last_mae'         : result[2]['mae'][-1].cpu().item() if torch.is_tensor(result[2]['mae'][-1]) else result[2]['mae'][-1],
                            'first_smape'      : result[1]['smape'].cpu().item() if torch.is_tensor(result[1]['smape']) else result[1]['smape'],
                            'last_smape'       : result[2]['smape'][-1].cpu().item() if torch.is_tensor(result[2]['smape'][-1]) else result[2]['smape'][-1],
                            'best_epochs'      : result[9],
                            'train_losses' : result[0][0],
                            'eval_pre': result[1],
                            'eval_post': result[2]
                        })
        
                        if not summarized:
                            result_dict.update({
                                'losses'           : [[v.cpu().item() if torch.is_tensor(v) else v for v in loss] for loss in result[0]],
                                'eval_results_pre' : {k: v.cpu().item() if torch.is_tensor(v) else v for k, v in result[1].items()},
                                'eval_results_post': {k: v.cpu().item() if torch.is_tensor(v) else v for k, v in result[2].items()},
                                })  
                        results = pd.concat([results, pd.DataFrame([result_dict])], ignore_index=True)
                    else:
                        print("Failed case")
                        # Attach possible errors
                        internal_errors['model_size'] = case['model_size']
                        internal_errors['n_epochs'] = case['n_epochs']
                        internal_errors['dataset_percent'] = case['dataset_percent']
                        internal_errors['maskared_percent'] = case['maskared_percent']
                        internal_errors['windows'] =  result[8]
                        internal_errors['windows'] = [result[8]]*len(internal_errors)
                        errors = pd.concat([errors, internal_errors])
                    if not error: mssg.print_error(f" case {case} | time: {result[4]}")
                    before = torch.cuda.memory_allocated()
                    model_case = None
                    gc.collect()
                    torch.cuda.empty_cache()
                    display(results)
                    if do_beep:
                        beep(1)
                    mssg.print(f"epoch {n_epochs}, dataset_percent {dataset_percent}, mask {maskared_percent}, sizes {sizes} -->")
                if save:
                    results.to_csv(file_results, index = False, header = True)
                    errors.to_csv(file_errors, index = False, header = True)
                if do_beep:
                    beep(2)
                    beep(2)
                mssg.print(f"epoch {n_epochs}, dataset_percent {dataset_percent}, mask {maskared_percent} -->")
            if do_beep:
                beep(3)
                beep(3)
                beep(3)
            mssg.print(f"epoch {n_epochs}, dataset_percent {dataset_percent}-->")
        if do_beep:
            beep(4)
            beep(4)
            beep(4)
            beep(4)
        mssg.print(f"epoch {n_epochs}-->")
    if do_beep:
        beep(1000)
        beep(1000)
        beep(1000)
        beep(1000)
        beep(1000)
    model_backup = None
    gc.collect()
    torch.cuda.empty_cache()
    return results, errors

In [15]:
n_epochs_list     = [5, 10, 20]
dataset_percents  = [0.25, 0.5, 0.75, 1] #1 No tendría sentido porque sería como hacer lo mismo que con mvp.. entrenar con todo el dataset.
maskared_percents = [0.25, 0.5, 0.75]
sizes             = [1, 5]
total_cases_small = len(n_epochs_list)*len(dataset_percents)*len(maskared_percents)*len(sizes)
print(f"Total cases: {total_cases_small}")

Total cases: 72


In [16]:
file_errors_small = "aux2"
file_results_small = "aux2"


In [17]:
print("Getting small artifact: ", enc_artifact_small_name)
enc_artifact_small = wandb_api.artifact(enc_artifact_small_name, type='learner')
print(enc_artifact_small.name)
moment_small = enc_artifact_small.to_obj()

Getting small artifact:  mi-santamaria/deepvats/zeroshot-moment-small-embedding:v0
zeroshot-moment-small-embedding:v0


wandb: Downloading large artifact zeroshot-moment-small-embedding:v0, 144.63MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.6


In [18]:
n_epochs_list_small    = [5]
dataset_percents_small = [0.25, 0.5]
maskared_percents_small= [0.25, 0.5, 0.75]
sizes_small            = [1, 5]
total_cases_small      = len(n_epochs_list_small)*len(dataset_percents_small)*len(maskared_percents_small)*len(sizes_small)
print(f"Total cases: {total_cases_small}")

Total cases: 12


In [27]:
r, e = cases_loop(
    model             = moment_small, 
    n_epochs_list     = n_epochs_list_small, 
    dataset_percents  = dataset_percents_small, 
    maskared_percents = maskared_percents_small, 
    n_sizes_list      = sizes_small, 
    verbose           = 1,
    summarized        = True,
    save              = True,
    file_errors       = file_errors_small,
    file_results      = file_results_small
)

0.25
--> epoch 5, dataset_percent 0.25, mask 0.25
 sizes 1
Cuda memmory allocated: 0
0.75



  0% 0/21 [00:00<?, ?it/s]
  5% 1/21 [00:01<00:32,  1.64s/it]
 10% 2/21 [00:02<00:17,  1.11it/s]
 14% 3/21 [00:02<00:11,  1.52it/s]
 19% 4/21 [00:02<00:09,  1.84it/s]
 24% 5/21 [00:03<00:07,  2.07it/s]
 29% 6/21 [00:03<00:06,  2.26it/s]
 33% 7/21 [00:03<00:05,  2.37it/s]
 38% 8/21 [00:04<00:05,  2.46it/s]
 43% 9/21 [00:04<00:04,  2.52it/s]
 48% 10/21 [00:04<00:04,  2.58it/s]
 52% 11/21 [00:05<00:03,  2.59it/s]
 57% 12/21 [00:05<00:03,  2.61it/s]
 62% 13/21 [00:06<00:03,  2.64it/s]
 67% 14/21 [00:06<00:02,  2.66it/s]
 71% 15/21 [00:06<00:02,  2.64it/s]
 76% 16/21 [00:07<00:01,  2.65it/s]
 81% 17/21 [00:07<00:01,  2.67it/s]
 86% 18/21 [00:07<00:01,  2.68it/s]
 90% 19/21 [00:08<00:00,  2.69it/s]
 95% 20/21 [00:08<00:00,  2.69it/s]
100% 21/21 [00:09<00:00,  2.31it/s]

  0% 0/35 [00:00<?, ?it/s]
  3% 1/35 [00:00<00:12,  2.82it/s]
  6% 2/35 [00:00<00:10,  3.05it/s]
  9% 3/35 [00:00<00:10,  3.14it/s]
 11% 4/35 [00:01<00:09,  3.18it/s]
 14% 5/35 [00:01<00:09,  3.20it/s]
 17% 6/35 [00:01<00:09

[1]  [ cases_loop ]  case {'model_size': 'small', 'n_epochs': 5, 'dataset_percent': 0.25, 'maskared_percent': 0.25, 'n_windows': 1, 'windows': None} | time: 10.961773872375488


,model_size,n_epochs,dataset_percent,maskared_percent,n_windows,time,first_train_loss,first_mse,first_rmse,first_mae,...,last_rmse,last_mae,last_smape,windows,first_eval_loss,last_eval_loss,best_epochs,train_losses,eval_pre,eval_post
0,small,5,0.25,0.25,1,10.961774,0.02889,0.047907,0.002344,0.027334,...,0.030187,0.083044,0.594351,[12],0.002258,0.029017,[4],"[0.028889855636017665, 0.017953439482620785, 0.018182241890047277, 0.021169555706105063, 0.01645758149347135]","{'mse': 0.04790733754429103, 'rmse': 0.0023439819486526193, 'mae': 0.02733410545534408, 'smape': 0.24390868142339733, 'loss': 0.0022581913409729687}","{'mse': [0.14732370499094594], 'rmse': [0.030186870637853528], 'mae': [0.08304399845619853], 'smape': [0.5943511398788206], 'loss': [0.02901685922794665]}"


[1]  [ cases_loop ] epoch 5, dataset_percent 0.25, mask 0.25, sizes 1 -->
--> epoch 5, dataset_percent 0.25, mask 0.25
 sizes 5
Cuda memmory allocated: 168909824
0.75



  0% 0/21 [00:00<?, ?it/s]
  5% 1/21 [00:01<00:27,  1.38s/it]
 10% 2/21 [00:01<00:15,  1.26it/s]
 14% 3/21 [00:02<00:10,  1.65it/s]
 19% 4/21 [00:02<00:08,  1.93it/s]
 24% 5/21 [00:02<00:07,  2.13it/s]
 29% 6/21 [00:03<00:06,  2.27it/s]
 33% 7/21 [00:03<00:05,  2.36it/s]
 38% 8/21 [00:04<00:05,  2.44it/s]
 43% 9/21 [00:04<00:04,  2.49it/s]
 48% 10/21 [00:04<00:04,  2.53it/s]
 52% 11/21 [00:05<00:03,  2.54it/s]
 57% 12/21 [00:05<00:03,  2.57it/s]
 62% 13/21 [00:05<00:03,  2.59it/s]
 67% 14/21 [00:06<00:02,  2.60it/s]
 71% 15/21 [00:06<00:02,  2.61it/s]
 76% 16/21 [00:07<00:01,  2.62it/s]
 81% 17/21 [00:07<00:01,  2.60it/s]
 86% 18/21 [00:07<00:01,  2.58it/s]
 90% 19/21 [00:08<00:00,  2.58it/s]
 95% 20/21 [00:08<00:00,  2.59it/s]
100% 21/21 [00:09<00:00,  2.32it/s]

  0% 0/35 [00:00<?, ?it/s]
  3% 1/35 [00:00<00:10,  3.21it/s]
  6% 2/35 [00:00<00:10,  3.21it/s]
  9% 3/35 [00:00<00:09,  3.20it/s]
 11% 4/35 [00:01<00:09,  3.23it/s]
 14% 5/35 [00:01<00:09,  3.22it/s]
 17% 6/35 [00:01<00:08

KeyboardInterrupt: 

In [28]:
import torch.autograd.profiler as profiler

In [29]:
n_epochs_list_small    = [5]
dataset_percents_small = [0.25]#, 0.5]
maskared_percents_small= [0.25]#, 0.5, 0.75]
sizes_small            = [1]#, 5]
total_cases_small      = len(n_epochs_list_small)*len(dataset_percents_small)*len(maskared_percents_small)*len(sizes_small)
print(f"Total cases: {total_cases_small}")

Total cases: 1


In [30]:
with profiler.profile(use_cuda = True) as prof:
    with profiler.record_function("model_inference"):
        r, e = cases_loop(
            model             = moment_small, 
            n_epochs_list     = n_epochs_list_small, 
            dataset_percents  = dataset_percents_small, 
            maskared_percents = maskared_percents_small, 
            n_sizes_list      = sizes_small, 
            verbose           = 1,
            summarized        = True,
            save              = True,
            file_errors       = file_errors_small,
            file_results      = file_results_small
        )
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

0.25
--> epoch 5, dataset_percent 0.25, mask 0.25
 sizes 1
Cuda memmory allocated: 320816640
0.75




  0% 0/21 [00:00<?, ?it/s]

  5% 1/21 [00:00<00:09,  2.19it/s]

 10% 2/21 [00:00<00:08,  2.24it/s]

 14% 3/21 [00:01<00:08,  2.22it/s]

 19% 4/21 [00:01<00:07,  2.26it/s]

 24% 5/21 [00:02<00:06,  2.29it/s]

 29% 6/21 [00:02<00:06,  2.30it/s]

 33% 7/21 [00:03<00:06,  2.30it/s]

 38% 8/21 [00:03<00:05,  2.31it/s]

 43% 9/21 [00:03<00:05,  2.31it/s]

 48% 10/21 [00:04<00:04,  2.32it/s]

 52% 11/21 [00:04<00:04,  2.30it/s]

 57% 12/21 [00:05<00:03,  2.29it/s]

 62% 13/21 [00:05<00:03,  2.31it/s]

 67% 14/21 [00:06<00:03,  2.32it/s]

 71% 15/21 [00:06<00:02,  2.32it/s]

 76% 16/21 [00:06<00:02,  2.33it/s]

 81% 17/21 [00:07<00:01,  2.33it/s]

 86% 18/21 [00:07<00:01,  2.31it/s]

 90% 19/21 [00:08<00:00,  2.29it/s]

 95% 20/21 [00:08<00:00,  2.30it/s]

100% 21/21 [00:09<00:00,  2.30it/s]


  0% 0/35 [00:00<?, ?it/s]

  3% 1/35 [00:00<00:12,  2.73it/s]

  6% 2/35 [00:00<00:12,  2.74it/s]

  9% 3/35 [00:01<00:11,  2.76it/s]

 11% 4/35 [00:01<00:11,  2.74it/s]

 14% 5/35 [00:01<00:11,  2.70

[1]  [ cases_loop ]  case {'model_size': 'small', 'n_epochs': 5, 'dataset_percent': 0.25, 'maskared_percent': 0.25, 'n_windows': 1, 'windows': None} | time: 12.92174768447876



 83% 29/35 [07:40<01:35, 15.89s/it]


,model_size,n_epochs,dataset_percent,maskared_percent,n_windows,time,first_train_loss,first_mse,first_rmse,first_mae,...,last_rmse,last_mae,last_smape,windows,first_eval_loss,last_eval_loss,best_epochs,train_losses,eval_pre,eval_post
0,small,5,0.25,0.25,1,12.921748,0.069952,0.212211,0.062612,0.107959,...,0.004402,0.034754,0.352938,[12],0.002416,0.004232,[3],"[0.06995183285159458, 0.028591394424438477, 0.0139179910267038, 0.007644833431446126, 0.011108384101784654]","{'mse': 0.21221099460062284, 'rmse': 0.06261189140802231, 'mae': 0.10795903062464951, 'smape': 0.5754837913490757, 'loss': 0.002416278217597643}","{'mse': [0.06401244115026496], 'rmse': [0.004402437600306676], 'mae': [0.03475445105227127], 'smape': [0.35293762837882403], 'loss': [0.004231672743030469]}"


[1]  [ cases_loop ] epoch 5, dataset_percent 0.25, mask 0.25, sizes 1 -->


[1]  [ cases_loop ] epoch 5, dataset_percent 0.25, mask 0.25 -->


[1]  [ cases_loop ] epoch 5, dataset_percent 0.25-->


[1]  [ cases_loop ] epoch 5-->


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference        91.44%       45.801s        99.83%       50.001s       50.001s       45.816s        91.47%       50.008s       50.008s             1  
                                     CheckpointFunction         0.91%     456.376ms         2.91%        1.459s       5.212ms     459.638ms         0.92%        1.461s       5.218ms           280  
         

In [31]:
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference        91.44%       45.801s        99.83%       50.001s       50.001s       45.816s        91.47%       50.008s       50.008s             1  
                                     CheckpointFunction         0.91%     456.376ms         2.91%        1.459s       5.212ms     459.638ms         0.92%        1.461s       5.218ms           280  
         